In [6]:
import torch
import numpy as np
import math

In [8]:
def gen_B(S1, S2, S3):  # 第一组  β即N_REAL  +++++求shitf++++
    M = (S1 * S2) / S3
    M.squeeze()
    M=M.cpu()#先将tensor放到cpu上
    M = M.numpy()
    
    daxiao = S2.shape[0]  # 第一层权重的shape[0]是32 shape[0]表示行数 是一维大小位32的列向量
    SCALE = np.zeros(daxiao, dtype=np.uint32, order='C')  # 相当于32个输出通道 每个对应一组shift
    N_REAL = np.zeros(daxiao, dtype=np.uint32, order='C')
    for i, ii in enumerate(M):  # enumerate() 函数用于将一个可遍历的数据对象(如列表、元组或字符串)组合为一个索引序列，同时列出数据和数据下标

        while not (ii >= 0.5 and ii <= 1.0):  # 左移到（0.5，1） 左移一次相当于*2
            ii *= 2
        pass
        mmmm = ii * (2 ** 32)  # 乘2^32

        SCALE[i] = mmmm.astype(np.int32)

    for i, ii in enumerate(M):
        N_REAL[i] = round(math.log(SCALE[i] / ii, 2)) - 32   # fpga加了1这里要减1,  β值也是m维 相当于存的mmmm
        if int(N_REAL[i])<0:
            print("error\n")
        if N_REAL[i] !=0:
            N_REAL[i]=N_REAL[i]-1

    return N_REAL


def gen_M_N(S1, S2, S3):  # 第二组求M'即M  ++++++返回scale和shift+++++
    daxiao = S2.shape[0]
    M = np.zeros(daxiao, dtype=np.uint32, order='C')
    N_REAL = gen_B(S1, S2, S3)
    M = np.zeros(S2.shape[0])
    
    for i, ii in enumerate(M):
        Scale=(S1*S2[i]/S3).cpu()
        # Scale=Scale.numpy
        M[i] = (torch.round(Scale * (2 ** (32 + N_REAL[i] + 1)))).numpy()  # s1s2/s3 *2^(32+β+1)
    M = M.astype(np.uint32)
    # exit()
    return M, N_REAL


# r_b=s1*s2*q_b
# r_b是量化前的bias,q_b是量化后的bias
def gen_int_bias(s1, s2, bias_float):  # 求bias/s1s2即bb
    aa = bias_float / s1
    # print(bias_float)
    # exit()
    bb = torch.div(aa, s2)  # 对应元素做除法

    # for i, m in enumerate(bb):
    #     bb[i] = round(m.item())
    # bias = bb.int()
    return bb


def gen_M(s1, s2, s3):
    aa = s1 * s2
    M = aa / s3
    return M


def new_bias(z1, q2, bias):  # 求最终的bias=bias/s1s2-q2z1
    q2 = q2.type(torch.float64)
    bias1 = z1 * q2
    shape = bias1.shape
    n_bias = np.zeros(shape[0], dtype=np.float64, order='C')
    for m in range(shape[0]):  # bias1的维度是M C K K 将C K K 做累加 变成M维
        n_bias[m] = bias1[m, :, :, :].sum()  # 从第一组开始一直有m组，m是输出通道数
        # print()
        n_bias[m] = (bias[m] - n_bias[m])  # 做减法
    # print(n_bias) 第一层n_bias是一维32个
    # exit()
    daxiao = shape[0]  # 第一层是32
    SCALE = np.zeros(daxiao, dtype=np.float64, order='C')
    # N_REAL = np.zeros(daxiao, dtype=np.float32, order='C')
    N_REAL = []
    for i, ii in enumerate(n_bias):  # i和ii就是从n_bias中取值
        index = 0

        while not (abs(ii) >= (2 ** 23) and abs(ii) <= (2 ** 24)):
            if index >= 16:  # fpga里面最多移动16位,所有成到16就停止了,这样精度也够了
                break
            else:
                ii *= 2
                index = index + 1

        N_REAL.append(index)
        SCALE[i] = round(ii)
    out_bias = []
    for index in range(shape[0]):
        data_integer_old = ('{:024b}'.format(int(SCALE[index]) & 0xffffff))  # {:024b} 24位2二进制不足补0；& 0xffffff按位与
        n = N_REAL[index]
        symbol = '0'
        if n_bias[index] < 0:  # 符号位
            symbol = '1'#这里也不是多此一举，因为如果你给一个uint24，MSB=1，那么你在FPGA那边如何分辨这是正数还是负数，所以还是需要1bit的符号位来帮助fpga做判断的
        elif n_bias[index] > 0:
            symbol = '0'
        data_integer = data_integer_old[8:]
        data_decimal = '{:07b}'.format(int(n))
        out_bias1 = symbol + str(data_decimal) + str(data_integer_old)  # 1bit+7bit+24bit
        a = int(out_bias1, 2)  # 转成int型 out_bias1为二进制 ；a是十进制
        out_bias.append(a)  # 一个一个写入out_bias
    # print(out_bias)
    # exit()
    return out_bias


def get_add_bias(new, shape, old):  # 补0
    for kernel_num in range(shape):
        new[kernel_num] = old[kernel_num]
    return new


def get_add_SCALE(new, shape, old):  # 补0
    for kernel_num in range(shape):
        new[kernel_num] = old[kernel_num]
    return new


def get_add_NREAL(new, shape, old):  # 补0
    for kernel_num in range(shape):
        new[kernel_num] = old[kernel_num]
    return new


def get_weight(new_weight, shape, weight, inchannel):  # 八入八出操作 输入通道inchannel不固定 输出通道outchannel为8
    j = 0

    shift_num = 0
    for index in range(inchannel):
        if (inchannel == (1 << index)):  # index不停左移一位直到与inchannel相等 来判断移了几位   2的shift_num次方=inchannel
            shift_num = index
            break
    # if shape[0] == 32 and shape[1] == 64 and shape[2] == 1:
    #     # print("shape:",new_weight.shape)
    #     with open('weight_1x1_8i8o(2).txt', 'a')as f:
    #         f.write('-' * 40 + '\n')
    #         f.write(str(shape) + '\n')
    for i in range(shape[2]):  # mckk  K
        for ii in range(shape[3]):  # K
            for kernel_times in range(shape[0] >> 3):  # >>右移三位 因为输出通道outchannel默认为8 卷积核个数
                for channel_in_times in range(shape[1] >> shift_num):  # 右移shift_num 输入通道数
                    for iii in range(8):  # shape[0] >> 3 右移了3次即2^3 要补8次
                        for iiii in range(
                                inchannel):  # shape[1] >> shift_num 右移了shift_num即2^shift_num 要补inchannel次(2^shift_num =inchannel)
                            # print('++++++++++++++++++')
                            weight[j] = new_weight[kernel_times * 8 + iii][channel_in_times * inchannel + iiii][i][ii]
                            # if shape[0] == 32 and shape[1] == 64 and shape[2] == 1:
                            # with open('weight_1x1_8i8o(2).txt', 'a')as f:
                            #     f.write(str(kernel_times * 8 + iii) + ',')
                            #     f.write(str(channel_in_times * inchannel + iiii) + ',')
                            #     f.write(str(i) + ',')
                            #     f.write(str(ii) + '\n')
                            #
                            j += 1
    # exit()
    return weight


def add_weight_channel(new_weig, weig, shape):  # 补0 把weig存入new_weig；new_weig是全0，
    for kernel_num in range(shape[0]):
        for channel_in_num in range(shape[1]):
            for row in range(shape[2]):
                for col in range(shape[3]):
                    new_weig[kernel_num][channel_in_num][row][col] = weig[kernel_num][channel_in_num][row][col]
    return new_weig


def tensorr(x):
    tensor_py = torch.from_numpy(np.load(x))  # 创建tensor
    return tensor_py


def get_weight2(new_weight, shape, weight):  # 四维权重写成一维
    j = 0
    for kernel_times in range(shape[0]):
        for channel_in_times in range(shape[1]):
            for i in range(shape[2]):
                for ii in range(shape[3]):
                    # print('++++++++++++++++++')
                    weight[j] = new_weight[kernel_times][channel_in_times][i][ii]
                    j += 1
    return weight


In [11]:
S1=torch.rand(1)*0.499+0.001
S2=torch.rand(768)*0.499+0.001
S3=torch.rand(1)*0.499+0.001
bias=torch.rand(1)*0.499+0.001
SCALE, N_REAL = gen_M_N(S1, S2, S3)
SCALE
N_REAL


C:\Users\25073\AppData\Local\Temp\ipykernel_12988\3006563256.py:13: RuntimeWarning: overflow encountered in double_scalars
  ii *= 2


In [2]:
int('1010010',2)

82

In [1]:
Bias_Tmp='1111'
BiasAdd=int(Bias_Tmp, 2) - (1 << len(Bias_Tmp))
BiasAdd

-1

In [29]:
a=bin(10)
print(a)
print(len(a))
print('-0b'+a[2:5])
print(int('-0b'+a[2:5],2))
# bin(-5)
# for i in range(len(a)):
#     print(i,"  ",a[i])
# print(a[2:])

0b1010
6
-0b101
-5


In [34]:
a=bin(-127)
print(a)
print(a[3:].zfill(4))

-0b1111111
1111111


In [55]:
import torch
from torch.nn import functional as F

x = torch.rand(1, 8, 224, 224)
w = torch.rand(768, 8, 3, 3)
stride = (1, 1)

output = F.conv2d(x, w, stride=stride)
print(output.shape)

torch.Size([1, 768, 222, 222])


In [ ]:
a=int(102)
bb=torch.zeros()

In [60]:
a=torch.rand(1,768,14,14)
b=torch.rand(768)
torch.mul(a[0,:,1,1],b)

tensor([1.5007e-01, 2.2411e-03, 2.0470e-01, 5.2534e-03, 8.0235e-01, 5.2113e-01,
        1.6234e-01, 7.2542e-01, 1.4723e-01, 5.8487e-02, 3.2948e-04, 2.1061e-01,
        5.0463e-02, 4.2238e-02, 3.2456e-01, 1.0675e-02, 1.2187e-01, 1.3574e-01,
        3.3166e-01, 3.6028e-02, 2.1759e-01, 1.7268e-01, 1.4460e-01, 3.8965e-01,
        5.8133e-01, 1.1793e-01, 2.4376e-01, 3.6589e-01, 2.2602e-02, 1.4659e-01,
        3.5235e-01, 2.8605e-01, 5.2180e-01, 8.3473e-02, 3.1087e-01, 5.4658e-04,
        1.1127e-01, 1.1466e-02, 4.5347e-01, 1.5913e-01, 7.9129e-02, 2.1646e-01,
        1.9854e-02, 5.8259e-01, 2.3040e-01, 2.0833e-02, 1.2947e-01, 5.8352e-01,
        1.1117e-01, 5.3672e-01, 2.3853e-01, 6.7361e-02, 7.0189e-01, 4.8092e-01,
        5.0899e-01, 2.8234e-02, 1.5773e-01, 7.4047e-02, 2.4848e-01, 3.8188e-01,
        4.2239e-01, 1.8467e-02, 5.7272e-02, 3.8750e-01, 8.0112e-02, 5.1387e-01,
        1.8160e-01, 1.7189e-01, 5.6067e-01, 1.1905e-01, 4.3684e-01, 6.1986e-02,
        2.3197e-01, 3.6373e-01, 6.9991e-

In [83]:
import numpy as np
import torch
A=torch.tensor(0.8)
A=A.numpy()
A=A*(2**32)
A.astype(np.uint32)

3435973888

In [85]:
M=torch.rand(10)
for i in enumerate(M):  # enumerate() 函数用于将一个可遍历的数据对象(如列表、元组或字符串)组合为一个索引序列，同时列出数据和数据下标
    ii=M[i]

IndexError: too many indices for tensor of dimension 1

In [4]:
aa='00001001100000111010100011010110'
aa
len(aa[0:8])

8

In [2]:
import numpy as np

def convert_to_binary(number, num_bits):
    binary = bin(number & int("1" * num_bits, 2))[2:]
    return '0' * (num_bits - len(binary)) + binary

number = -1024
num_bits = 80

binary_representation = convert_to_binary(number, num_bits)
print(binary_representation)

11111111111111111111111111111111111111111111111111111111111111111111110000000000


In [3]:
bin(-1024)


'-0b10000000000'

In [78]:
import torch
def complement2Dec(binary):
    if binary[0] == '1':
        # 对负数进行补码处理
        inverted_binary = ''.join('1' if bit == '0' else '0' for bit in binary)
        decimal = -(int(inverted_binary, 2) + 1)
    else:
        # 对正数直接转换为十进制
        decimal = int(binary, 2)
    return decimal

aa=-1*torch.randint(-10,20,(1,))

aa_bin=bin(aa)
if aa<0:
    aa_bin=aa_bin[3:]
else:
    aa_bin=aa_bin[2:]
bb=int(aa_bin, 2) - (1 << len(aa_bin))
print("[error0]",aa,bb)
bb=complement2Dec(aa_bin)
print("[error1]",aa,bb)
if aa_bin[0]=='1':
    bb=int('-0b'+aa_bin,2)
    print("[right]",aa,bb)
else:
    bb=int(aa_bin,2)
    print("[right]",aa,bb)


[error0] tensor([3]) -1
[error1] tensor([3]) -1
[right] tensor([3]) -3


In [9]:
import torch
from torch.nn import functional as F
input=torch.zeros(1,2,3,3)
weight=torch.zeros(2,2,3,3)
bias=torch.ones(2)
output=F.conv2d(input, weight, stride=(1,1),bias=bias)
print(output)

tensor([[[[1.]],

         [[1.]]]])


In [22]:
bin(-15771904)


'-0b111100001010100100000000'

In [21]:
len("111100001010100100000000")

24

In [67]:
def fliplr(string):
    # string = "Hello, World!"
    reversed_string = string[::-1]
    return reversed_string
    # print(reversed_string)
    # 运行以上代码，将输出反转后的字符串："!dlroW ,olleH"。通过使用切片操作符[::-1]，可以从字符串的末尾开始，每次递减一个索引，从而实现字符串的反转。
def compbin2hex(binary):
    #二进制补码转十进制
    output=0
    if binary[0]=='1':#如果为负数
        inverted_binary = ''.join('1' if bit == '0' else '0' for bit in binary[1:])
        output=-(int(inverted_binary,2)+1)
    else:
        binary=fliplr(binary)
        for i in range(len(binary)):
            output=output+(ord(binary[i])-48)*pow(2,i)
    return output
output=compbin2hex('01010')
output

10

In [64]:
binary=bin(-127)
binary=binary[3:]
print(binary)
inverted_binary = ''.join('1' if bit == '0' else '0' for bit in binary[1:])
dec=int(inverted_binary,2)+1
dec

1111111


1

In [4]:
def convert_to_binary(number, num_bits):
    binary = bin(number & int("1" * num_bits, 2))[2:]
    return '0' * (num_bits - len(binary)) + binary
a=convert_to_binary(-129,9)
a

'101111111'

In [6]:
import torch

Bias_Tmp_All=torch.rand(768)
q1q2=torch.rand(4,768,14,14)
result=q1q2.permute(0,3,2,1)+Bias_Tmp_All
result[0,0,0,]-(q1q2[0,:,0,0]+Bias_Tmp_All)
result=result.permute(0,3,2,1)


SCALE=torch.rand(768)
ScaleMul=torch.mul(result.permute(0,3,2,1),SCALE)


In [10]:
a='00001111111111111111'
a[-17:]

'01111111111111111'

In [7]:
import torch
def Generate_Weight_Bin(Tensor,Type,Path):
    #生成权重和图片的bin文件
    #Tensor:输入的tensor
    #Type：可选：Image，ConvWeight，LinearWeight，QuantData
    #path:存储路径
    
    #image要求image是量化后的8bit数据
    if Type=="Image":
        assert len(Tensor.shape)==3
        with open (Path+"/Image.bin",'w') as ff:
        #Wq的维度：[OC,IC,K,K]
            for Row in range(Tensor.shape[1]):#遍历行
                for Col in range(Tensor.shape[2]):#遍历列
                    for IC in range(0,Tensor.shape[0],8):#遍历通道
                        Hex0=Tensor[IC+0,Row,Col].item()&0xff
                        Hex1=Tensor[IC+1,Row,Col].item()&0xff
                        Hex2=Tensor[IC+2,Row,Col].item()&0xff
                        Hex3=Tensor[IC+3,Row,Col].item()&0xff
                        Hex4=Tensor[IC+4,Row,Col].item()&0xff
                        Hex5=Tensor[IC+5,Row,Col].item()&0xff
                        Hex6=Tensor[IC+6,Row,Col].item()&0xff
                        Hex7=Tensor[IC+7,Row,Col].item()&0xff
                        #print('%02x%02x%02x%02x%02x%02x%02x%02x'%(Hex7,Hex6,Hex5,Hex4,Hex3,Hex2,Hex1,Hex0))
                        ff.write('%02x%02x%02x%02x%02x%02x%02x%02x'%(Hex7,Hex6,Hex5,Hex4,Hex3,Hex2,Hex1,Hex0))
                        ff.write("\n")
            ff.close()
Image=torch.randint(0,255,(8,224,224))
len(Image.shape)
print("Image"=='Image')
Generate_Weight_Bin(Image,'Image',r"E:\Transformer\Transformer_Arithmatic\Transformer_Main\BinPath")


True


In [2]:
AA=["S1","Z1"]
AA

['S1', 'Z1']

In [4]:
import torch
weight=torch.rand(4,2,3,3)
weight_permute=weight.permute(0,2,3,1)


weight_permute
WeightMatrix=torch.zeros(4,2*3*3)
for Oc in range(4):
    for H in range(3):
        for W in range(3):
            for Ic in range(2):
                print("%.4f"%weight[Oc,Ic,H,W].item(),end=' ')
                # print(weight_permute[Oc,H,W,Ic]==weight[Oc,Ic,H,W])
    print()
print(weight_permute.reshape(-1,2*3*3))
print(weight.reshape(-1,2*3*3))
print(weight)

0.5900 0.8746 0.0381 0.2223 0.0573 0.7255 0.5468 0.8782 0.6533 0.1052 0.5575 0.5817 0.2564 0.7570 0.5693 0.8580 0.1442 0.5555 
0.4053 0.2857 0.6090 0.4987 0.5069 0.6957 0.3159 0.4980 0.7419 0.4554 0.3331 0.0074 0.2225 0.6998 0.9449 0.7807 0.3040 0.8826 
0.2615 0.3207 0.4010 0.6163 0.4286 0.5436 0.7966 0.4408 0.6621 0.4182 0.0859 0.9937 0.1960 0.0945 0.2836 0.8827 0.0148 0.0588 
0.6993 0.9975 0.8634 0.0371 0.1230 0.0656 0.8094 0.8603 0.3696 0.2882 0.1645 0.9922 0.6486 0.7259 0.4661 0.0603 0.5087 0.1564 
tensor([[0.5900, 0.8746, 0.0381, 0.2223, 0.0573, 0.7255, 0.5468, 0.8782, 0.6533,
         0.1052, 0.5575, 0.5817, 0.2564, 0.7570, 0.5693, 0.8580, 0.1442, 0.5555],
        [0.4053, 0.2857, 0.6090, 0.4987, 0.5069, 0.6957, 0.3159, 0.4980, 0.7419,
         0.4554, 0.3331, 0.0074, 0.2225, 0.6998, 0.9449, 0.7807, 0.3040, 0.8826],
        [0.2615, 0.3207, 0.4010, 0.6163, 0.4286, 0.5436, 0.7966, 0.4408, 0.6621,
         0.4182, 0.0859, 0.9937, 0.1960, 0.0945, 0.2836, 0.8827, 0.0148, 0.0588],
   

In [11]:
import scipy.io as sio
data={}
data["data"]=[1,2,3]

sio.savemat(r'matlab\Embedding_Conv.mat', data)


In [2]:
print(weight_permute.shape)

torch.Size([4, 3, 2, 3])


In [7]:
import torch
weight=torch.rand(4,2,3,3)
weight_fattened=weight.permute(0,2,3,1).reshape(weight.shape[0],-1)
weight_cat=x=torch.cat((torch.zeros(weight.shape[0],3),weight_fattened),dim=1)
print(weight_fattened)
weight_cat

tensor([[0.6203, 0.6479, 0.5679, 0.7791, 0.9912, 0.5173, 0.0303, 0.2098, 0.6674,
         0.2378, 0.6693, 0.1406, 0.1802, 0.7911, 0.1496, 0.9827, 0.1039, 0.0095],
        [0.9029, 0.3299, 0.7488, 0.9962, 0.8423, 0.3409, 0.0522, 0.7883, 0.2416,
         0.1972, 0.8414, 0.5133, 0.9008, 0.3382, 0.0687, 0.4170, 0.9094, 0.6457],
        [0.6904, 0.6208, 0.4632, 0.8345, 0.7178, 0.5761, 0.3408, 0.5039, 0.3442,
         0.8730, 0.4708, 0.8199, 0.9649, 0.0668, 0.9774, 0.8267, 0.6873, 0.3437],
        [0.4341, 0.3287, 0.1510, 0.1694, 0.4085, 0.4417, 0.8256, 0.2875, 0.0944,
         0.3703, 0.1294, 0.5641, 0.5131, 0.1129, 0.1789, 0.3293, 0.6852, 0.6605]])


tensor([[0.0000, 0.0000, 0.0000, 0.6203, 0.6479, 0.5679, 0.7791, 0.9912, 0.5173,
         0.0303, 0.2098, 0.6674, 0.2378, 0.6693, 0.1406, 0.1802, 0.7911, 0.1496,
         0.9827, 0.1039, 0.0095],
        [0.0000, 0.0000, 0.0000, 0.9029, 0.3299, 0.7488, 0.9962, 0.8423, 0.3409,
         0.0522, 0.7883, 0.2416, 0.1972, 0.8414, 0.5133, 0.9008, 0.3382, 0.0687,
         0.4170, 0.9094, 0.6457],
        [0.0000, 0.0000, 0.0000, 0.6904, 0.6208, 0.4632, 0.8345, 0.7178, 0.5761,
         0.3408, 0.5039, 0.3442, 0.8730, 0.4708, 0.8199, 0.9649, 0.0668, 0.9774,
         0.8267, 0.6873, 0.3437],
        [0.0000, 0.0000, 0.0000, 0.4341, 0.3287, 0.1510, 0.1694, 0.4085, 0.4417,
         0.8256, 0.2875, 0.0944, 0.3703, 0.1294, 0.5641, 0.5131, 0.1129, 0.1789,
         0.3293, 0.6852, 0.6605]])

In [5]:
import torch
import numpy as np
image=torch.randint(0,255,(1,8,224,224))
def Generate_Bin(Tensor,Type,Path):
    #生成权重和图片的bin文件
    #Tensor:输入的tensor
    #Type：可选：Image，ConvWeight，LinearWeight，QuantData
    #path:存储路径
    if Type=="Image":
        Output_Reshape1=torch.permute(Tensor,[0,2,3,1])
        Output_Reshape2=torch.reshape(Output_Reshape1,[-1])
        Output_Reshape3 = Output_Reshape2.numpy()
        int_output= Output_Reshape3.astype(np.uint8)#转为int，因为之前的tensor是float
        print(int_output.dtype)
        ff=open (Path+"/Image.bin",'w')
        int_output.tofile(ff)
        ff.close()
    elif Type=="ConvWeight":
        Output_Reshape1=torch.permute(Tensor,[0,2,3,1])
        Output_Reshape2=torch.reshape(Output_Reshape1,[-1])
        Output_Reshape3 = Output_Reshape2.numpy()
        int_output= Output_Reshape3.astype(np.uint8)#转为int，因为之前的tensor是float
        print(int_output.dtype)
        ff=open (Path+"ConvWeight.bin",'w')
        int_output.tofile(ff)
        ff.close()

uint8


In [7]:
image[0,:,0,0:3]

tensor([[173, 235, 130],
        [201, 241, 134],
        [ 84,  64, 219],
        [  0,  23,  45],
        [206,  32,  22],
        [107,  36, 151],
        [ 90, 250,  62],
        [221, 106, 204]])

: 

In [2]:
import numpy as np
a=np.zeros((1,3),np.uint8)
b=np.ones((1,3),np.uint8)
c=[a,b]
c[1]

array([[1, 1, 1]], dtype=uint8)

In [3]:
import numpy as np
import torch
Tensor=[np.array([1,1,1,1]), np.array([2,2,2,2]), np.array([3,3,3,3])]
Bias=Tensor[0].astype(np.uint32)
Scale=Tensor[1].astype(np.uint32)
Shift=Tensor[2].astype(np.uint16)
ff=open ("BinPath/ConvQuant.bin",'a')
Bias.tofile(ff)
Scale.tofile(ff)
Shift.tofile(ff)
ff.close()

In [5]:
import torch
import numpy as np
Tensor=[np.array([1,1,1,1]), np.array([2,2,2,2]), np.array([3,3,3,3]),torch.rand(1,2)]
Tensor

[array([1, 1, 1, 1]),
 array([2, 2, 2, 2]),
 array([3, 3, 3, 3]),
 tensor([[0.5326, 0.7909]])]

In [10]:
import numpy as np
a=[1,2,3]
b=np.array(a)
b

array([1, 2, 3])

In [1]:
import torch
A=torch.zeros(1,3,224,224)+117
print(A)
B=torch.rand(1,5,224,224)
C=torch.cat((A,B),dim=1)
print(C.shape)

tensor([[[[117., 117., 117.,  ..., 117., 117., 117.],
          [117., 117., 117.,  ..., 117., 117., 117.],
          [117., 117., 117.,  ..., 117., 117., 117.],
          ...,
          [117., 117., 117.,  ..., 117., 117., 117.],
          [117., 117., 117.,  ..., 117., 117., 117.],
          [117., 117., 117.,  ..., 117., 117., 117.]],

         [[117., 117., 117.,  ..., 117., 117., 117.],
          [117., 117., 117.,  ..., 117., 117., 117.],
          [117., 117., 117.,  ..., 117., 117., 117.],
          ...,
          [117., 117., 117.,  ..., 117., 117., 117.],
          [117., 117., 117.,  ..., 117., 117., 117.],
          [117., 117., 117.,  ..., 117., 117., 117.]],

         [[117., 117., 117.,  ..., 117., 117., 117.],
          [117., 117., 117.,  ..., 117., 117., 117.],
          [117., 117., 117.,  ..., 117., 117., 117.],
          ...,
          [117., 117., 117.,  ..., 117., 117., 117.],
          [117., 117., 117.,  ..., 117., 117., 117.],
          [117., 117., 117.,  ...

In [1]:
import torch
import torch.nn as nn

class MyNetwork(nn.Module):
    def __init__(self):
        super(MyNetwork, self).__init__()
        self.fc1 = nn.Linear(in_features=10, out_features=20)
        self.fc2 = nn.Linear(in_features=20, out_features=30)

    def forward(self, x):
        x = self.fc1(x)
        layer1_name = self.fc1.__class__.__name__
        print("Layer 1 Name:", layer1_name)
        x = self.fc2(x)
        layer2_name = self.fc2.__class__.__name__
        print("Layer 2 Name:", layer2_name)
        return x

# 创建网络实例
net = MyNetwork()

# 创建输入数据
input_data = torch.randn(5, 10)  # 假设输入数据维度为(5, 10)

# 前向传播
output = net(input_data)

Layer 1 Name: Linear
Layer 2 Name: Linear


In [1]:
a=10
str(a)

'10'